# TSA Chapter 6: Case Study - Stationarity Tests

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch6/TSA_ch6_case_stationarity/TSA_ch6_case_stationarity.ipynb)

ADF tests on levels and differences of interest rates.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download interest rate data for cointegration case study
try:
    gs10 = web.DataReader('GS10', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    gs2 = web.DataReader('GS2', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    rates = pd.DataFrame({'10Y': gs10.iloc[:,0], '2Y': gs2.iloc[:,0]}).dropna()
except:
    np.random.seed(42); n = 408; dates = pd.date_range('1990-01-01', periods=n, freq='MS')
    rw = np.cumsum(np.random.normal(0, 0.1, n)) + 5
    rates = pd.DataFrame({'10Y': rw + np.random.normal(0, 0.2, n), '2Y': rw - 0.5 + np.random.normal(0, 0.15, n)}, index=dates)
print(f'Rates data: {len(rates)} obs')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 7))
for i, (col, c) in enumerate(zip(rates.columns, [BLUE, RED])):
    adf = adfuller(rates[col].dropna()); axes[0,i].plot(rates.index, rates[col], color=c, lw=1)
    p_color = RED if adf[1] > 0.05 else GREEN; axes[0,i].set_title(f'{col} Level (ADF p={adf[1]:.3f})', fontweight='bold', color=p_color)
    diff = rates[col].diff().dropna(); adf_d = adfuller(diff)
    axes[1,i].plot(diff.index, diff.values, color=c, lw=0.8); axes[1,i].axhline(0, color=GRAY, ls='--')
    axes[1,i].set_title(f'{col} Diff (ADF p={adf_d[1]:.4f})', fontweight='bold', color=GREEN)
plt.tight_layout(); save_chart(fig, 'ch6_case_stationarity'); plt.show()